## プロンプト管理

必要なライブラリをダウンロードします。

In [ ]:
%pip install -r ../requirements.txt

実行に必要な環境変数を読み込みます。

In [2]:
import os, warnings

# おまじない
warnings.filterwarnings("ignore")

endpoint = "http://localhost:3000"
public_key = os.getenv("PUBLIC_KEY")
secret_key = os.getenv("SECRET_KEY")

SDKからプロンプトを作成します。

In [3]:
from langfuse import Langfuse

langfuse = Langfuse(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint
)

In [4]:

langfuse.create_prompt(
    name="test-sdk",
    type="text",
    prompt="{{topic}}について教えて",
    labels=["sdk"],
    config={},
)

SDKからプロンプトを更新します。

In [5]:
langfuse.create_prompt(
    name="test-sdk",
    type="text",
    prompt="簡潔に{{topic}}について教えて",
    labels=["sdk"],
    config={},
)

SDKからプロンプトを利用するために `production` ラベルを付与します。

In [6]:
langfuse.create_prompt(
    name="test-sdk",
    type="text",
    prompt="簡潔に{{topic}}について教えて",
    labels=["sdk", "production"],
    config={},
)

SDKからプロンプトを利用します。

In [7]:
prompt = langfuse.get_prompt(
    name="test-sdk"
)

print(prompt.prompt)

簡潔に{{topic}}について教えて


変数に値を埋め込みます。

In [8]:
prompt.compile(topic="カルビクッパ")

'簡潔にカルビクッパについて教えて'

LangChainのPromptTemplateとして使うための関数も用意されています。

In [9]:
prompt.get_langchain_prompt()

'簡潔に{topic}について教えて'

（オプション）プロンプトとトレースを紐づける

In [10]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langfuse.decorators import observe, langfuse_context
from langfuse.model import ModelUsage

@observe(as_type="generation")
def ask_the_bigbaby(query: str):
    if os.getenv("OPENAPI_API_KEY") == None:
        from langchain_cohere.chat_models import ChatCohere
        cohere_api_key = os.getenv("COHERE_API_KEY")
        model = "command-r-plus"
        chat = ChatCohere(cohere_api_key=cohere_api_key, model=model)
    else:
        from langchain_openai.chat_models import ChatOpenAI
        openai_api_key = os.getenv("OPENAI_API_KEY")
        model = "gpt-4o-mini"
        chat = ChatOpenAI(api_key=openai_api_key, model=model)
    langfuse_managed_prompt = langfuse.get_prompt(name="test-sdk", version=3)
    
    chain = (
        {"topic": RunnablePassthrough()} |
        PromptTemplate.from_template(template=langfuse_managed_prompt.get_langchain_prompt()) |
        chat
    )
    response = chain.invoke(query)
    langfuse_context.update_current_observation(
        prompt=langfuse_managed_prompt,
        model=model
    )
    return response

langfuse_context.configure(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint
)
ask_the_bigbaby("お茶漬け")

AIMessage(content='お茶漬けは、ご飯に茶をかけた日本の伝統的な料理です。お茶は通常、緑茶や番茶が使われます。お好みで、様々な具材を乗せることができ、人気の具材としては、梅干し、鮭、たらこ、昆布などがあります。お茶漬けは、簡単に作れて、さっぱりとした味わいから、日本人のソウルフードとして愛されています。', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '99183a56-d412-4af9-b320-569464561e03', 'token_count': {'input_tokens': 208.0, 'output_tokens': 101.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '99183a56-d412-4af9-b320-569464561e03', 'token_count': {'input_tokens': 208.0, 'output_tokens': 101.0}}, id='run-89163564-155f-4b15-ad27-66f8d23dfae0-0', usage_metadata={'input_tokens': 208, 'output_tokens': 101, 'total_tokens': 309})